PROJECT TEAM ID: PTID-CDS-JAN-24-1761
PROJECT CODE :  Buisness Case for PRCL-0015
PROJECT NAME :Bank GoodCredit

# Business Case
    Bank GoodCredit wants to predict cred score for current credit card customers. The cred score will denote a customer’s credit worthiness and help the bank in reducing credit default risk.
                           Target variable → Bad_label
                        -->0 – Customer has Good credit history
                        -->1 – Customer has Bad credit history (falls into 30 DPD + bucket)
                           Our benchmark model has gini as 37.9.
                                               

## Domain Analysis
                                         
Bank credit analysis involves verifying and determining the creditworthiness of a potential client by looking at their financial state, credit reports, and business cash flows.The goal of credit analysis is to determine the level of default risk that a client presents to the company and the losses that the bank will suffer if the client defaults.The risk level that a client presents determines whether the bank will approve or reject the loan application, and if approved, the amount to be awarded.some of the major table this data has and every tables has there own important columns which are Customers Account, Customers Demographics and Customers Enquiry.                                         

* Cust_Account :- This table contains customer’s historical accounts data and payments history.
* Cust_Demographics :- This table contains customer’s historical enquiry data such as enquiry amount and enquiry purpose.
* Cust_Enquiry :- Current customer applications with demographic data
* __Note__ that demographics features are renamed as features and obscured in accordance with privacy policies.

* I will explain every necesseary columns when i'll do __Data Cleaning__.


In [52]:
!pip install pymysql  
!pip install mysql-connector



In [53]:
import os               # Handling the Current Working Directory
import mysql.connector  #  mysql-connector for making connections to database server
import pandas as pd     # import pandas for analyzing, cleaning, exploring, and manipulating data
import numpy as np      # import numpy for working with mathematices part/numerical data
## visualisation
import matplotlib.pyplot as plt
import seaborn as sns
# for ignoring warning
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,classification_report,f1_score


In [54]:
connection=mysql.connector.connect(host="18.136.157.135",
                                  user="dm_team1",
                                  password="DM!$Team&279@20!")

InterfaceError: 2003: Can't connect to MySQL server on '18.136.157.135:3306' (10065 A socket operation was attempted to an unreachable host)

In [ ]:
cursor=connection.cursor()
cursor.execute('show databases')
for i in cursor:
    print(i)



In [ ]:
connection=mysql.connector.connect(host="18.136.157.135",
                                  user="dm_team1",
                                  password="DM!$Team&279@20!",
                                  database="project_banking")

In [ ]:
# getting all tables of bank goodcredit
db_tables=pd.read_sql_query('show tables',connection)
print(db_tables)

In [ ]:
pd.read_sql_query('show tables',connection)

In [ ]:
# Replace 'your_query_here' with your actual SQL query
# Fetch data from MySQL and store it in a DataFrame
Cust_Account=pd.read_sql_query('select* from Cust_Account',connection)
Cust_Demographics=pd.read_sql_query('select * from Cust_Demographics',connection)
Cust_Enquiry=pd.read_sql_query('select * from Cust_Enquiry',connection)

In [ ]:
Cust_Account

In [ ]:
data = pd.merge(Cust_Account,Cust_Demographics,on ="customer_no",how="inner")

In [ ]:
data.to_csv("Bank_GoodCredit",index=False)

In [ ]:
data

# Basic Checks

In [ ]:
data.info()

In [ ]:
data.head(5)

In [ ]:
data.tail()

In [ ]:
data.describe()

In [ ]:
data.describe(include="O")

# Data Preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
df = []
for i in data.columns:
    if (data[i].isnull().sum())/len(data)*100>=40:
        df.append(i)

In [ ]:
print(df)

### Droping columns which has >=40% missing value

In [ ]:
data.drop(['closed_dt', 'amt_past_due', 'paymenthistory2', 'creditlimit', 'cashlimit', 'rateofinterest', 'paymentfrequency',
           'actualpaymentamount', 'feature_8', 'feature_9', 'feature_10', 'feature_13', 'feature_17', 'feature_18',
           'feature_45', 'feature_48', 'feature_49', 'feature_51', 'feature_53', 'feature_57', 'feature_61', 'feature_73',
           'feature_74'],axis=1,inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data

### Now we are going to drop unnecessary columns

In [ ]:
data.drop(["reporting_dt","paymenthistory1","dt_opened_y","entry_time","feature_2","feature_20","feature_21",
          "feature_22","feature_24","feature_46","feature_47","feature_54","feature_75","feature_77","last_paymt_dt","paymt_str_dt","paymt_end_dt","feature_5","feature_6","feature_62","feature_70",
           "feature_79","feature_39","dt_opened_x","upload_dt"],axis=1,inplace=True)

### Now, i'm going to drop that columns which has no variety in values

In [ ]:
data.drop(["owner_indic","feature_11","feature_19","feature_23","feature_25","feature_31","feature_33","feature_42",
          "feature_55","feature_58","feature_59","feature_60","feature_67","feature_76","feature_78","feature_52",],axis=1,inplace=True)

In [ ]:
pd.reset_option('display.max_rows',None)

In [ ]:
data.isnull().sum()  # checking null value

In [ ]:
!pip install missingno


In [ ]:
import missingno as msno
## this plot is used to detect the null values
msno.matrix(data,color=(0.60,0.300,0.300),fontsize=20)

In [ ]:
for i in data.columns:
    print(f"======= { i } ========\n")
    print(data[i].value_counts())

## Filling all null value by the help of Measure of Central Tendency

In [ ]:
data.loc[data["opened_dt"].isnull(),"opened_dt"] = "13-Apr-12"
data.loc[data["high_credit_amt"].isnull(),"high_credit_amt"] = np.mean(data.high_credit_amt)
data.loc[data["feature_1"].isnull(),"feature_1"] = "Platinum Maxima"
data.loc[data["feature_3"].isnull(),"feature_3"] = 682.0
data.loc[data["feature_4"].isnull(),"feature_4"] = 3.000000
data.loc[data["feature_7"].isnull(),"feature_7"] = np.mean(data.feature_7)
data.loc[data["feature_12"].isnull(),"feature_12"] = "PM1"
data.loc[data["feature_14"].isnull(),"feature_14"] = 12.000000
data.loc[data["feature_15"].isnull(),"feature_15"] = "SA03"
data.loc[data["feature_16"].isnull(),"feature_16"] = "AS03"
data.loc[data["feature_26"].isnull(),"feature_26"] = 0.000000
data.loc[data["feature_27"].isnull(),"feature_27"] = "Graduate"
data.loc[data["feature_28"].isnull(),"feature_28"] = "New Delhi"
data.loc[data["feature_29"].isnull(),"feature_29"] = np.mean(data.feature_29)
data.loc[data["feature_30"].isnull(),"feature_30"] = 2010.0
data.loc[data["feature_32"].isnull(),"feature_32"] = "Self"
data.loc[data["feature_34"].isnull(),"feature_34"] = 2.0
data.loc[data["feature_35"].isnull(),"feature_35"] = 42759.59392727169
data.loc[data["feature_36"].isnull(),"feature_36"] = "Private Ltd. Co."
data.loc[data["feature_37"].isnull(),"feature_37"] = "Banking/Financial Services"
data.loc[data["feature_38"].isnull(),"feature_38"] = "MANAGER"
data.loc[data["feature_40"].isnull(),"feature_40"] = 0.0
data.loc[data["feature_41"].isnull(),"feature_41"] = 11.0
data.loc[data["feature_43"].isnull(),"feature_43"] = "New Delhi"
data.loc[data["feature_44"].isnull(),"feature_44"] = np.mean(data.feature_44)
data.loc[data["feature_50"].isnull(),"feature_50"] = "Y"
data.loc[data["feature_63"].isnull(),"feature_63"] = "2010-0"
data.loc[data["feature_64"].isnull(),"feature_64"] = 10.0
data.loc[data["feature_65"].isnull(),"feature_65"] = 157.0
data.loc[data["feature_66"].isnull(),"feature_66"] = np.mean(data.feature_66)
data.loc[data["feature_68"].isnull(),"feature_68"] = 1.0
data.loc[data["feature_69"].isnull(),"feature_69"] = np.mean(data.feature_69)
data.loc[data["feature_71"].isnull(),"feature_71"] = 10.0
data.loc[data["feature_72"].isnull(),"feature_72"] = "R"
data.loc[data["feature_56"].isnull(),"feature_56"] = 10.0

In [ ]:
data.isnull().sum()  # checking null value

In [ ]:
import missingno as msno ## this plot is used to detect the null values
msno.matrix(data,color=(0.60,0.300,0.300),fontsize=20)

In [ ]:
def convert_into_datetime(columns):
    data[columns]=pd.to_datetime(data[columns])
    
# Converting the columns into Datetime using for loop
for i in ['opened_dt']:  
    convert_into_datetime(i) 
    
# Create the new columns with month 
data['opened_dt_year']=data['opened_dt'].dt.year

data.drop('opened_dt',axis=1,inplace=True)

In [ ]:
# here i'm going to split the feature_63 columns for getting only year

data.feature_63 = data['feature_63'].str.split('-',expand=True)[0]

In [ ]:
data

In [ ]:
#Change columns type according to their value some columns having object type but it should in int type 

data.high_credit_amt = data.high_credit_amt.astype(int)
data.feature_3 = data.feature_3.astype(int)
data.feature_4 = data.feature_4.astype(int)
data.feature_14 = data.feature_14.astype(int)
data.feature_26 = data.feature_26.astype(int)
data.feature_30 = data.feature_30.astype(int)
data.feature_34 = data.feature_34.astype(int)
data.feature_40 = data.feature_40.astype(int)
data.feature_44 = data.feature_41.astype(int)
data.feature_56 = data.feature_56.astype(int)
data.feature_63 = data.feature_63.astype(int)
data.feature_64 = data.feature_64.astype(int)
data.feature_65 = data.feature_65.astype(int)
data.feature_68 = data.feature_68.astype(int)
data.feature_71 = data.feature_71.astype(int)

In [ ]:
data

## Exploratory Data Analysis

In [ ]:
catagorical=data.describe(include="O")
catagorical

In [ ]:
numeric=data.describe(exclude="O")
numeric

In [ ]:
sns.countplot(x='feature_1', hue='Bad_label', data=data)
plt.title("Credit Cards vs Bad Labels")
plt.xticks(rotation=90)
plt.show()

### Insights

* Most of the person having Platinum Maxima and Platinum Deligh Cards.
* person who has Golf Card they having 50-50 chance of good or bad Credit Score (bad or good labels).
* person who have RBL bank Fun+,Insignia and Platinum Cricke Cards they have perfect credit score

In [ ]:
sns.histplot(data.feature_3)# asign histplot for feature_3

In [ ]:
sns.countplot(x='feature_27',hue = 'Bad_label', data=data)#assign countplot between feature_27 and Bad Labels
plt.title("customers education vs Bad Labels")# assign title name of distributions
plt.xticks(rotation=90)#increasing font of x-axis and rorating 90 degree 
plt.show()

In [ ]:
sns.countplot(x='feature_28',hue = 'Bad_label',data=data)#assign countplot between feature_28 and Bad Labels
plt.title("Customer State vs Bad Labels") # assign title name of distributions
plt.xticks(fontsize = 7,rotation = 90)#increasing font of x-axis and rorating 90 degree 
plt.yticks(fontsize = 11)#increasing font of y-axis
plt.show()

### Insights

* Most of the Customers belong from New Delhi.

In [ ]:
sns.countplot(x='feature_32',hue ='Bad_label',data=data)#assign countplot between feature_32 and Bad Labels
plt.title("Customers stay vs Bad Labels") # assign title name of distributions
plt.xticks(rotation=90)#increasing font of x-axis and rorating 90 degree 
plt.show()

In [ ]:
sns.countplot(x='feature_36',hue = 'Bad_label',data=data) #assign countplot between feature_36 and Bad Labels
plt.title("customers Business vs Bad Labels") # assign title name of distributions
plt.xticks(rotation=90)#increasing font of x-axis and rorating 90 degree 
plt.show()

In [ ]:
sns.countplot(x='feature_37',hue = 'Bad_label',data=data) #assign countplot between feature_37 and Bad Labels
plt.title("customers Ocupations vs Bad Labels") # assign title name of distributions
plt.xticks(rotation=90)#increasing font of x-axis and rorating 90 degree 
plt.show()

## Final Conclusions of Insights :-


* Most of the person having Platinum Maxima and Platinum Deligh Cards.
* Person who has Golf Card they having 50-50 chance of good or bad Credit Score (bad or good labels).
* Person who have RBL bank Fun+,Insignia and Platinum Cricke Cards they have perfect credit score.
* Most of the customers have good credit score.
* Few chance of that customers who has completed Post Graduate, Graduate and MBA/MMS their Credit score are Bad.
* Most of the Customers belong from New Delhi.
* Most of the Customer having Credit Cards who belong from Private Ltd.Co.
* Customers who started their bussiness in Partership they maintain their credit score.
* Most chance of that Customers who doing job in Banking/Financial services they taking credit cards.
* Most of the customers who has crdit cards they stay at their on home.

# Checking Outliers

In [ ]:
# getting that columns which type are integers
data1= data.select_dtypes(include='int')

In [ ]:

data1

In [ ]:
plt.figure(figsize=(100,50))
plotnumber=1
for i in data1:
    plt.subplot(5,4,plotnumber)
    sns.boxplot(x=data[i])
    plotnumber=plotnumber+1
    plt.xticks(rotation=45)
    plt.xlabel(i,fontsize=20)
plt.tight_layout()

In [ ]:
data1.columns

### Insights
*  By using boxplot we find out the outliers Columns
*  Columns are 
               1)acct_type
               2)high_credit_amt
               3)cur_balance_amt
               4)feature_65
* The outliers are filling the median values 

In [ ]:
def find_boundaries(data,variables):
    Q1=data[variables].quantile(0.25)
    Q3=data[variables].quantile(0.75)
    IQR=Q3-Q1
    lowerboundary=Q1-1.5*IQR
    upperboundary=Q3+1.5*IQR
    return lowerboundary,upperboundary

In [ ]:
lower_acct_type,upper_acct_type=find_boundaries(data,'acct_type')
print('upper bondary for acct_type',upper_acct_type)
print('lower boundary for acct_type',lower_acct_type)

In [ ]:
data.loc[data['acct_type']>16.0,'acct_type']=np.median(data.acct_type)

In [ ]:
lower_high_credit_amt,upper_high_credit_amt=find_boundaries(data,'high_credit_amt')
print('upper bondary for high_credit_amt',upper_high_credit_amt)
print('lower boundary for high_credit_amt',lower_high_credit_amt)

In [ ]:
data.loc[data['high_credit_amt']>301662.5,'high_credit_amt']=np.median(data.high_credit_amt)
data.loc[data['high_credit_amt']<-142597.5,'high_credit_amt']=np.median(data.high_credit_amt)

In [ ]:
lower_cur_balance_amt,upper_cur_balance_amt=find_boundaries(data,'cur_balance_amt')
print('upper bondary for cur_balance_amt',upper_cur_balance_amt)
print('lower boundary for cur_balance_amt',lower_cur_balance_amt)

In [ ]:
data.loc[data['cur_balance_amt']>52666.25,'cur_balance_amt']=np.median(data.cur_balance_amt)
data.loc[data['cur_balance_amt']<-31599.75,'cur_balance_amt']=np.median(data.cur_balance_amt)

In [ ]:
lower_feature_3,upper_feature_3=find_boundaries(data,'feature_3')
print('upper bondary for feature_3',upper_feature_3)
print('lower boundary for feature_3',lower_feature_3)

In [ ]:
data.loc[data['feature_3']>825.0,'feature_3']=np.median(data.feature_3)
data.loc[data['feature_3']<609.0,'feature_3']=np.median(data.feature_3)

In [ ]:
lower_feature_65,upper_feature_65=find_boundaries(data,'feature_65')
print('upper bondary for feature_65',upper_feature_65)
print('lower boundary for feature_65',lower_feature_65)

In [ ]:
data.loc[data['feature_65']>370.0,'feature_65']=np.median(data.feature_65)
data.loc[data['feature_65']<-198.0,'feature_65']=np.median(data.feature_65)

In [ ]:
plt.figure(figsize=(100,50))
plotnumber=1
for i in data1:
    plt.subplot(5,4,plotnumber)
    sns.boxplot(x=data[i])
    plotnumber=plotnumber+1
    plt.xticks(rotation=45)
    plt.xlabel(i,fontsize=20)
plt.tight_layout()

## Insights 
* Finally we remove the outliers
* Remaning data point are not outliers ,They are near the maximum values,minimum value

## Conversion of categorical variables

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
def encoding(data):
    le=LabelEncoder()
    for i in data.select_dtypes('object'):
        data[i]=le.fit_transform(data[i])
    return data

In [ ]:
data=encoding(data)
data.head()

# Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
data[['customer_no','high_credit_amt','cur_balance_amt','feature_3','feature_7','feature_15','feature_16','feature_29','feature_30','feature_35','feature_38','feature_63','feature_66','feature_69','opened_dt_year']]=scaler.fit_transform(data[['customer_no','high_credit_amt','cur_balance_amt','feature_3','feature_7','feature_15','feature_16','feature_29','feature_30','feature_35','feature_38','feature_63','feature_66','feature_69','opened_dt_year']])

In [ ]:
data

# Feature Selection

In [ ]:
data.corr()  # checking corelation

In [ ]:
plt.figure(figsize=(28,28))#increase plot size
sns.heatmap(data.corr(),annot=True)

In [ ]:
data.drop(["feature_1","feature_63","feature_68","feature_69","feature_41"],axis=1,inplace=True) # droping columns

In [ ]:
plt.figure(figsize=(28,28))#increase plot size
sns.heatmap(data.corr(),annot=True)

# Insights
* Finally we droped higly correlated columns,here there is no correlation then moving to next step

In [ ]:
# Now selecting independent variable and dependent variables
x = data.drop("Bad_label",axis=1)
x

In [ ]:
y = data.Bad_label
y

In [ ]:
data.Bad_label.value_counts()

# Model Creation

In [ ]:
from sklearn.model_selection import train_test_split  # import train test split for spliting the data
# creating x_train,x_test,y_train,y_test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=43) 

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

# Data Balancing

In [ ]:
from imblearn.over_sampling import SMOTE  # assign SMOTE for handling unbalanced data
sm = SMOTE()

In [ ]:
x_sm,y_sm = sm.fit_resample(x_train,y_train)  # balancing data 

In [ ]:
from collections import Counter  # checking data balanced or not
print(Counter(y_train))
print(Counter(y_sm))

# Model Selection

# Logistic Regression Model
*Logistic Regression is a statistical technique used for binary classification, where the goal is to predict whether an instance belongs to one of two classes (usually represented as 0 or 1). Despite its name, logistic regression is used for classification tasks, not regression tasks. It models the probability that an instance belongs to the positive class, making it a probabilistic algorithm.

In [ ]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(x_sm,y_sm)
y_pred1=model1.predict(x_test)
y_pred_lr_train = model1.predict(x_train)


In [ ]:
cm=confusion_matrix(y_test,y_pred1)
recall=recall_score(y_test,y_pred1,average='weighted')
precision=precision_score(y_test,y_pred1,average='weighted')
f1score=f1_score(y_test,y_pred1,average='weighted')
accuracy=accuracy_score(y_test,y_pred1)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)


## Decision Tree Classifier

* A Decision Tree is a popular and intuitive machine learning algorithm used for both classification and regression tasks. It models data as a tree-like structure of decisions and their possible consequences. Each internal node of the tree represents a decision based on a feature, and each leaf node represents a prediction or a class label.

**Here's how a Decision Tree works:**

**Selecting the Best Feature:**

* The algorithm starts by selecting the feature that best splits the data into different classes or reduces the variance of the target variable. The selection criterion is typically based on measures like Gini impurity, entropy, or mean squared error, depending on whether the task is classification or regression.

**Splitting the Data:**

* The selected feature is used to split the data into subsets. Each subset corresponds to a unique value of the selected feature.

**Repeating the Process:**

* The above steps are recursively applied to the subsets created at each internal node. The algorithm keeps selecting features that provide the best splits, and the tree continues to grow until a stopping criterion is met.

**Stopping Criteria:**

* The tree growth can be stopped based on various criteria, such as:

Maximum depth: The maximum number of levels the tree can have.
Minimum samples per leaf: The minimum number of samples required to create a leaf node.
Minimum samples per split: The minimum number of samples required to perform a split.
Leaf Node Assignments:
Once the tree is built, each leaf node is assigned a class label (in the case of classification) or a predicted value (in the case of regression). The majority class or the mean/median value of the target variable in the leaf node's subset is typically used.

**Prediction:*
* To make a prediction for a new instance, the instance is traversed down the tree from the root to a leaf node. The predicted class label or value of the leaf node is then used as the final prediction.

**Advantages of Decision Trees:**

* Intuitive: Decision Trees are easy to understand and visualize, making them useful for explaining decisions to non-experts.
* Non-parametric: They can capture complex relationships without assuming a specific functional form.
* Feature Importance: Decision Trees can provide insights into feature importance and feature interactions.

**Limitations of Decision Trees:**

* Prone to Overfitting: Without proper pruning or limiting growth, Decision Trees can become overly complex and overfit the training data.
**Instability:**
* Small changes in the data can lead to different tree structures, making them sensitive to variations.

**Bias towards Dominant Classes:**

* In classification, they can favor dominant classes if the dataset is imbalanced.
* To address some of the limitations, ensemble techniques like Random Forest and Gradient Boosting are often used, which combine multiple Decision Trees to improve performance and robustness.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier()
model2.fit(x_sm,y_sm)
y_pred2=model2.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,classification_report,f1_score
cm=confusion_matrix(y_test,y_pred2)
recall=recall_score(y_test,y_pred2,average='weighted')
precision=precision_score(y_test,y_pred2,average='weighted')
f1score=f1_score(y_test,y_pred2,average='weighted')
accuracy=accuracy_score(y_test,y_pred2)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)

# Random Forest Classifier
* Random Forest is a powerful ensemble learning technique that combines multiple individual decision trees to create a more accurate and robust predictive model. It's commonly used for classification and regression tasks, offering improved performance over single decision trees by mitigating their limitations.

**Here's how Random Forest works:**

**Bootstrap Aggregating (Bagging):**

* Random Forest employs a technique called bagging, where multiple subsets of the training data are created through random sampling with replacement. Each subset is used to train a separate decision tree. This introduces diversity in the training data, reducing the likelihood of overfitting.


**Random Feature Selection:**

* In addition to using random subsets of data, Random Forest introduces randomness in feature selection during the construction of each individual tree. At each split, the algorithm considers only a subset of features, rather than all features. This helps to decorrelate the trees and improve generalization.

**Building Multiple Trees:**

* The Random Forest algorithm builds a specified number of decision trees using the bootstrapped subsets of data and the random feature subsets. Each decision tree is built using the standard Decision Tree algorithm (e.g., using Gini impurity or entropy for classification, or mean squared error for regression).

**Voting or Averaging:**

* For classification tasks, the final prediction of the Random Forest is determined through a majority vote among the individual decision trees. For regression tasks, predictions are averaged across the trees.

**Advantages of Random Forest:**

**Reduced Overfitting:**
* By aggregating multiple trees and introducing randomness, Random Forest reduces overfitting compared to individual decision trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier()
model3.fit(x_sm,y_sm)
y_pred3=model3.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,classification_report,f1_score
cm=confusion_matrix(y_test,y_pred3)
recall=recall_score(y_test,y_pred3,average='weighted')
precision=precision_score(y_test,y_pred3,average='weighted')
f1score=f1_score(y_test,y_pred3,average='weighted')
accuracy=accuracy_score(y_test,y_pred3)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)

# XGBRFClassifier

* XGBRFClassifier is a classifier based on the XGBoost algorithm that is specifically designed to work with random forests. XGBoost (Extreme Gradient Boosting) is a popular and powerful machine learning algorithm known for its performance and versatility in both classification and regression tasks. While XGBoost typically utilizes boosted trees, the XGBRFClassifier is a variation that focuses on the random forest ensemble structure. It is part of the XGBoost library and offers benefits similar to traditional random forest classifiers but with some enhancements from the XGBoost framework.

**Here's a breakdown of the key features and aspects of the XGBRFClassifier:**

**Ensemble Structure:**

* The XGBRFClassifier constructs an ensemble of decision trees, similar to traditional random forest classifiers. However, it leverages the techniques and optimizations of the XGBoost framework.

**Gradient Boosting Principles:**

* While the XGBRFClassifier uses random forest-style decision trees, it still benefits from some of the boosting principles of XGBoost. This includes gradient boosting, which iteratively improves the model by focusing on correcting the mistakes of previously added trees.

**Tree Building Process:**

* The classifier constructs decision trees using a random subset of features and a subset of the training data (bootstrap aggregation or bagging). This helps to introduce diversity and reduce overfitting.

**Regularization and Pruning:**

* Like XGBoost, the XGBRFClassifier applies regularization techniques to prevent overfitting. It also supports early stopping to avoid constructing too many trees that may not improve the model's performance.

**Hyperparameter Tuning:**

* The XGBRFClassifier comes with a variety of hyperparameters that can be tuned to optimize its performance for specific tasks. These hyperparameters control aspects like the depth of trees, learning rate, subsampling ratio, and more.

**Parallel and Distributed Computing:**

* The XGBoost library is designed to efficiently use parallel and distributed computing resources. This can lead to faster training times and improved scalability.

**Feature Importance and Visualization:**

* XGBRFClassifier provides tools for assessing feature importance, enabling you to understand which features contribute the most to the model's predictions.

* The XGBRFClassifier is particularly useful when you want to harness the benefits of both random forest-style ensembles and the optimizations and regularization techniques provided by XGBoost. It can be applied to a wide range of classification tasks, including those with complex relationships and high-dimensional data.






In [ ]:
from xgboost import XGBRFClassifier
model4=XGBRFClassifier() ##n_estimators = 500, seed = 123
model4.fit(x_sm,y_sm)
y_pred4=model4.predict(x_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred4)
recall=recall_score(y_test,y_pred4,average='weighted')
precision=precision_score(y_test,y_pred4,average='weighted')
f1score=f1_score(y_test,y_pred4,average='weighted')
accuracy=accuracy_score(y_test,y_pred4)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)


# GradientBoostingClassifier


* The GradientBoostingClassifier is a powerful and popular machine learning algorithm used for classification tasks. It is part of the gradient boosting family of algorithms, which are designed to improve the predictive accuracy of models by combining the predictions of multiple weak learners (typically decision trees) in an additive manner. The GradientBoostingClassifier is implemented in scikit-learn and is based on the gradient boosting framework.

**Here's an explanation of the key concepts and features of the GradientBoostingClassifier:**

**Gradient Boosting Framework:**

* Gradient boosting is an ensemble learning technique that builds an additive model in a forward stage-wise manner. Each stage constructs a new weak learner that corrects the errors made by the previous stage.

**Weak Learners (Decision Trees):**

* The base learners in the GradientBoostingClassifier are often shallow decision trees, also known as weak learners. These trees have limited depth and can be thought of as simple rules applied to the data.

**Boosting Iterations:**

* The algorithm performs a series of boosting iterations. At each iteration, a new decision tree is added to the ensemble to correct the mistakes made by the existing ensemble.

**Residuals and Gradient Descent:**

* The new tree is fitted to the negative gradient of the loss function with respect to the current predictions. This guides the tree construction to reduce the errors of the previous predictions.

**Learning Rate:**

* A learning rate parameter controls the contribution of each new tree to the ensemble. A lower learning rate can prevent overfitting by shrinking the impact of each new tree.

**Regularization:**

* The GradientBoostingClassifier employs regularization techniques to prevent overfitting, such as controlling the depth of the trees and using a minimum number of samples per leaf.

**Subsampling:**

* Random subsampling of the data can be applied to each boosting iteration to introduce randomness and improve generalization.

**Feature Importance:**

* The algorithm provides a measure of feature importance based on how often features are used in decision trees and how much they contribute to reducing the loss.

**Early Stopping:**

* To avoid overfitting, the algorithm can be stopped early based on the performance on a validation set. This prevents adding too many trees that may lead to overfitting.

* The GradientBoostingClassifier is effective for a wide range of classification tasks, including both binary and multi-class problems. It can capture complex relationships in data and often achieves competitive performance compared to other advanced algorithms. However, it may require tuning of hyperparameters to achieve optimal performance and to prevent overfitting.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model5=GradientBoostingClassifier()
model5.fit(x_sm,y_sm)
y_pred5=model5.predict(x_test)


In [ ]:
cm=confusion_matrix(y_test,y_pred5)
recall=recall_score(y_test,y_pred5,average='weighted')
precision=precision_score(y_test,y_pred5,average='weighted')
f1score=f1_score(y_test,y_pred5,average='weighted')
accuracy=accuracy_score(y_test,y_pred5)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)

# MLPClassifier- ANN

##### * MLPClassifier is a type of ANN (Artificial Neural Network). It is a multilayer perceptron (MLP) classifier, which is a feedforward neural network that can be used for classification tasks.

* An MLP consists of multiple layers of neurons, each layer is fully connected to the following one. The neurons in the input layer receive the input data, and the neurons in the output layer produce the output predictions. The neurons in the hidden layers perform the intermediate computations.

* The MLPClassifier class in scikit-learn implements a multi-layer perceptron (MLP) algorithm that trains using Backpropagation. It can be used for both binary and multi-class classification tasks.

* The MLPClassifier class has the following parameters:

              hidden_layer_sizes: A list of the number of neurons in each hidden layer.
              activation: The activation function for the hidden layers.
              solver: The algorithm used to train the model.
              alpha: The learning rate.
              max_iter: The maximum number of iterations.
              random_state: The random seed.

In [ ]:
from sklearn.neural_network import MLPClassifier
model6= MLPClassifier( hidden_layer_sizes=(100,50,2), activation='relu',solver='adam',
                       learning_rate_init=0.0001,
                       max_iter=150,random_state=78) ## model object creation max_iter=Stopping parameter
model6.fit(x_sm,y_sm) ## training the data
y_pred6=model6.predict(x_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred6)
recall=recall_score(y_test,y_pred6,average='weighted')
precision=precision_score(y_test,y_pred6,average='weighted')
f1score=f1_score(y_test,y_pred6,average='weighted')
accuracy=accuracy_score(y_test,y_pred6)
print("CM:",cm)
print("Recall:",recall)
print("Precision:",precision)
print("F1 Score:",f1score)
print("Accuracy:",accuracy)

## Cross Validation in All Algorithms
* Cross-validation is a fundamental technique in machine learning used to evaluate the performance of algorithms and models, as well as to tune hyperparameters and assess their generalization ability. It involves splitting the dataset into multiple subsets to simulate different training and testing scenarios, helping to ensure that the model's performance is representative and reliable.

In [ ]:
list_of_models=[model1,model2,model3,model4,model5,model6]

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy_cvs=[]
for i in list_of_models:
    cvs=cross_val_score(i,x,y,cv=5,scoring='accuracy')
    p=cvs.mean()
    accuracy_cvs.append(p)
accuracy_cvs   

In [ ]:
names_of_algorithms=['LogisticRegression','DecisionTreeClassifier','RandomForestClassifier','XGBRFClassifier','GradientBoostingClassifier','MLPClassifier']


In [ ]:
plt.figure(figsize=(20,15))
sns.barplot(x=names_of_algorithms,y=accuracy_cvs)
plt.tight_layout()

# **Finalizing the algorithm**

* Among all algorithms "GradientBoostingClassifier"aves the more accuracy.So finalized algorithm is radientBoostingClassifier

In [ ]:
import pickle as pkl
pkl.dump(model3,open("mobile.pkl","wb"))
read=pkl.load(open("mobile.pkl","rb"))

In [ ]:
independent_input_data=[0.258297,10,0.374619,0.366569,0.409302,3,0.083813,10,12,0.600551,0.570058,2,8,39,0.481703,0.647059,4,2,0.030000,6,1,0.464324,0,43,0,1,21,21,15,0.379517,21,1,0.888889]
e=read.predict([independent_input_data])[0]

In [ ]:
x_train

In [ ]:
data.iloc[166664]

# Conclusion
In conclusion, Bank GoodCredit appears to have a promising project on its hands. The bank's strategy is aligned with industry trends and market demands, and it has taken steps to mitigate potential risks and challenges. The project's focus on innovation and technology positions it well for future growth and competitiveness in the banking sector.

Bank GoodCredit's commitment to market analysis, financial projections, and compliance with regulatory requirements showcases a comprehensive approach to project planning. Additionally, its emphasis on marketing and customer acquisition indicates a strong intent to build a robust customer base.

However, the ultimate success of the project will depend on its execution, the effectiveness of the team, and the dynamic nature of the banking industry. Continuous monitoring and adaptation to changing market conditions will be crucial for achieving the projected financial goals and maintaining a positive reputation among customers and stakeholders.

It's important to note that this conclusion is based on a general hypothetical scenario. For a more accurate and detailed conclusion, specific project details and data would be required.